In [2]:
from abc import abstractmethod
import requests
import pandas as pd
from pandas import DataFrame
from datetime import datetime, timedelta
import time

In [22]:
class Post:
    def __init__(self) -> None:
        self.headers = {"User-Agent": "Mozilla/5.0"}

    def read(self, **params):
        # print(params)
        resp = requests.post(self.url, headers=self.headers, data=params)
        return resp.json()

    @property
    @abstractmethod
    def url(self):
        return NotImplementedError

In [23]:
class WebAPI(Post):    
    # read with period
    def read_period(self, start, end, **params):
        time.sleep(0.5)
        delta = timedelta(days=365)
        day = timedelta(days=1)
        if start > end: return
        elif start + delta < end:
            curr_end = start + delta
            params['strtDd'] = start.strftime("%Y%m%d")
            params['endDd'] = curr_end.strftime("%Y%m%d")
            result = super().read(**params)
            result['output'] = self.read_period(curr_end + day, end, **params)['output'] \
                                + result['output']
            return result
        else:
            params['strtDd'] = start.strftime("%Y%m%d")
            params['endDd'] = end.strftime("%Y%m%d")
            result = super().read(**params)
            return result
            
    def read(self, **params):
        result = None
        params.update(bld=self.bld)

        if not 'strtDd' in params or not 'endDd' in params:
            return super().read(**params)

        start_date = pd.to_datetime(params['strtDd'])
        end_date = pd.to_datetime(params['endDd'])
        return self.read_period(start_date, end_date, **params)
        
    @property
    @abstractmethod
    def bld(self):
        return NotImplementedError

    @property
    def url(self):
        return "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"

In [24]:
class 개별종목시세추이(WebAPI):
    def load(self, stock_code: str, start: str, end: str):
        result = self.read(isuCd=stock_code,
                            strtDd=start,
                            endDd=end)
        return DataFrame(result['output'])
    @property
    def bld(self):
        return "dbms/MDC/STAT/standard/MDCSTAT01701"

In [25]:
class 주식종목검색(WebAPI):
    def search(self, searchText: str):
        result = self.read(searchText=searchText,
                            mktsel='ALL',
                            typeNo='0')
        return DataFrame(result['block1'])
    @property
    def bld(self):
        return "dbms/comm/finder/finder_stkisu"

In [26]:
class KrxStock:
    def get_stock_code(self, stock_name: str):
        return 주식종목검색().search(stock_name)
    def get_stock_OHLCV(self, stock_code: str, start: str, end: str):
        return 개별종목시세추이().load(stock_code, start, end)

In [27]:
class 개별지수종합정보(WebAPI):
    def load(self, index_id: str, index_group_id, date: str) -> DataFrame:
        result = self.read(indIdx2=index_id,
                            indIdx=index_group_id,
                            trdDd=date)
        return DataFrame(result['output'])
    @property
    def bld(self):
        return "dbms/MDC/STAT/standard/MDCSTAT00601"

In [36]:
class KrxIndex:
    def __init__(self):
        self.index_list = {
            "KOSPI" : {"index_id": "001", "index_group_id": "1"}, 
            "KOSPI 200" : {"index_id": "028", "index_group_id": "1"}, 
            "KOSDAQ" : {"index_id": "001", "index_group_id": "2"}, 
            "KOSDAQ 150" : {"index_id": "203", "index_group_id": "2"}
        }

    def unpack_index_id(self, index_id: str, index_group_id: str):
            return index_id, index_group_id

    def get_index_constituents(self, index_name: str, date: str):
        try:
            index_id, index_group_id = self.unpack_index_id(**self.index_list[index_name])
        except:
            print(f"Select index in {[key for key in self.index_list.keys()]}")
            return None
        
        data = 개별지수종합정보().load(index_id, index_group_id, date)
        return data

    def get_index_constituents_OHLCV(self, index_name: str, start: str, end: str):
        start_date = pd.to_datetime(start)
        end_date = pd.to_datetime(end)
        delta = pd.to_timedelta('1 days')

        columns = ['STOCK_NAME', 'TRD_DD', 'TDD_CLSPRC', 'FLUC_TP_CD', 'CMPPREVDD_PRC', 'FLUC_RT', 'TDD_OPNPRC', \
                   'TDD_HGPRC', 'TDD_LWPRC', 'ACC_TRDVOL', 'ACC_TRDVAL', 'LIST_SHRS', 'MKTCAP']
        result = pd.DataFrame(columns=columns)

        i = 0
        while start_date <= end_date:
            data = self.get_index_constituents(index_name, start_date.strftime("%Y%m%d"))
            if data is None: continue
            for stock_name in data['ISU_ABBRV']:
                print(stock_name)
                full_code = KrxStock().get_stock_code(stock_name)['full_code'].iloc[0]
                curr = start_date.strftime("%Y%m%d")
                result.append(KrxStock().get_stock_OHLCV(full_code, curr, curr))
                result.at[-1, 'STOCK_NAME'] = stock_name
            start_date += delta
            if i == 10: break
            i += 1

        return result

In [37]:
class WebSource():
    def __init__(self):
        self.krxStock = KrxStock()
        self.krxIndex = KrxIndex()

In [38]:
ws = WebSource()
# print(ws.krxStock.get_stock_code('삼성전자'))
# print(ws.krxStock.get_stock_OHLCV('KR7005930003', '20230818', '20230818'))
# print(ws.krxIndex.get_index_constituents('KOSPI', '20230818'))
print(ws.krxIndex.get_index_constituents_OHLCV('KOSPI', '20230817','20230818'))

삼성전자
LG에너지솔루션
SK하이닉스
삼성바이오로직스
POSCO홀딩스
삼성SDI
LG화학
현대차
NAVER
포스코퓨처엠
기아
카카오
현대모비스
셀트리온
KB금융
삼성물산
신한지주
SK이노베이션
LG전자
포스코인터내셔널
삼성생명
LG
카카오뱅크
삼성화재
한국전력
KT&G
하나금융지주
HD현대중공업
삼성전기
메리츠금융지주
두산에너빌리티
삼성에스디에스
SK텔레콤
SK
하이브
고려아연
대한항공
HMM
한화오션
KT
기업은행
우리금융지주
S-Oil
HD한국조선해양
금양
크래프톤
삼성중공업
삼성엔지니어링
아모레퍼시픽
LG생활건강
SK바이오팜
SK아이이테크놀로지
현대글로비스
LG이노텍
SK스퀘어
한화솔루션
SK바이오사이언스
카카오페이
엔씨소프트
유한양행
롯데케미칼
DB손해보험
한화에어로스페이스
두산밥캣
한온시스템
한국타이어앤테크놀로지
오리온
맥쿼리인프라
한미반도체
LG디스플레이
CJ제일제당
HD현대
코스모신소재
현대제철
한국항공우주
LG유플러스
현대오토에버
미래에셋증권
F&F
한미약품
현대건설
넷마블
LS
삼성카드
GS
NH투자증권
SKC
현대미포조선
금호석유
호텔신라
삼성증권
강원랜드
현대로템
코웨이
한진칼
BGF리테일
농심
LS ELECTRIC
한국금융지주
영원무역
쌍용C&E
씨에스윈드
HD현대일렉트릭
한화시스템
팬오션
키움증권
롯데지주
GS리테일
한미사이언스
현대해상
아모레G
한전기술
롯데에너지머티리얼즈
DB하이텍
휠라홀딩스
한국가스공사
BNK금융지주
제일기획
CJ
이수페타시스
에스원
영풍제지
이마트
HL만도
효성첨단소재
롯데쇼핑
신세계
대우건설
한화생명
한솔케미칼
한화
HD현대인프라코어
JB금융지주
효성중공업
동서
LIG넥스원
KCC
OCI홀딩스
현대엘리베이
CJ대한통운
두산
동원산업
두산퓨얼셀
롯데정밀화학
에스디바이오센서
코스맥스
솔루엠
한전KPS
현대백화점
코스모화학
에스엘
오뚜기
현대위아
대한전선
SK네트웍스
KG모빌리티
LX세미콘
대덕전자
코오롱인더
덴티움
한국콜마
HD현대건설기계
효성티앤씨
금호타이어
삼아알미늄
녹십자
한샘
하이트진로
LX인터내셔널
TCC스